## Import libraries

In [ ]:
import glob

# import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import itertools
import sqlite3

import matplotlib.pyplot as plt
import pandas as pd

from tqdm.auto import tqdm

import pathlib
import mapclassify

import geopandas as gpd
from shapely.geometry import Polygon

import numpy as np

## Import geodata files

In [ ]:
shared_input_path = "/fp/projects01/ec85/models/max/special_issue/shared_input/"

In [ ]:
resultspath = pathlib.Path(
    "/cluster/work/projects/ec85/joint-wind/model-aggregated/resultscosts/"
)

In [ ]:
nuts_to_ssb = {
    "NO081": "NO03",
    "NO082": "NO30",
    "NO020": "NO34",
    "NO091": "NO38",
    "NO092": "NO42",
    "NO0A1": "NO11",
    "NO0A2": "NO46",
    "NO0A3": "NO15",
    "NO060": "NO50",
    "NO071": "NO18",
    "NO074": "NO54",
}

In [ ]:
desired_regions = ['NO03', 'NO11', 'NO15', 'NO18', 'NO30', 'NO34', 'NO38', 'NO42',
       'NO46', 'NO50', 'NO54']

In [ ]:
rectx1 = -12
rectx2 = 44
recty1 = 33
recty2 = 81

In [ ]:
polygon = Polygon(
    [
        (rectx1, recty1),
        (rectx1, recty2),
        (rectx2, recty2),
        (rectx2, recty1),
        (rectx1, recty1),
    ]
)
#europe = gpd.clip(europe, polygon)

In [ ]:
norway = (
    gpd
    .read_file(shared_input_path + 'geodata/onshore/shapes/NUTS_RG_10M_2021_4326.geojson')
    .replace(nuts_to_ssb)
    .query("NUTS_ID == @desired_regions")
    .loc[:,['NUTS_ID','geometry']]
    .rename(columns={'NUTS_ID' : 'index'})
    .set_index('index')
    .sort_index()
)
norway = norway.clip(norway, polygon)

In [ ]:
colordict = {
    "GasCCS": "#b20101",
    "Gas": "#d35050",
    "HydroRes": "#08ad97",
    "HydroRoR": "#4adbc8",
    "OnWind": "#235ebc",
    "Solar": "#f9d002",
    "Import": "#8a1caf",
    "Windoffshore": "#6895dd",
    "Lion": "#baf238",
    "PumpedHydro": "#51dbcc",
}

In [ ]:
compressionfileending = ".zstd"
compressiondict = {"method": "zstd", "level": 19, "threads": -1}

In [ ]:
def modelstatus(modelpath):
    logfile = modelpath / "highres.log"
    gamsfile = modelpath / "results.gdx"
    database = modelpath / "results.db"

    if logfile.is_file() and gamsfile.is_file() and database.is_file():
        with open(logfile) as logfile:
            if "Optimal solution found" in logfile.read():
                return "optimal"
            else:
                return "exists"
    else:
        return "missing"

# Reading files

In [ ]:
requested_scenarios = pd.read_csv(
    resultspath / "scenarios.csv", sep="\t", keep_default_na=False, index_col=0
).set_index("index").rename_axis(index={"index":""}).assign(
    path=lambda df: (
        resultspath
        / "models"
        / df.years.astype(str)
        / (
            df.naturs.astype(str)
            + "_"
            + df.faunas.astype(str)
            + "_"
            + df.samis.astype(str)
            + "_"
            + df.neighs.astype(str)
            + "_"
            + df.solars.astype(str)
            + "_"
            + df.spatials.astype(str)
            + "_"
            + df.cutoffs_wind.astype(str)
        )
    ),
    status=lambda df: df.path.apply(modelstatus),
)

# Did the scenarios solve optimally?
requested_scenarios.query("status != 'optimal'")

In [ ]:
def makenice(scenariolist, parameter):
    returnlist = []

    basicindexlist = ["year", "Nature", "Fauna", "Sami", "Neigh", "Solar", "Spatial","Cutoff"]

    if parameter == "cost":
        table = "scalarvariables"
        index = ["name"]
        newindex = ["variable"]
        extraindex = newindex + ["level_1"]
        value = "costs"
        types = {}
        tabletype = "multiple"

    if parameter == "cap":
        table = "var_tot_pcap"
        index = ["g"]
        newindex = ["technology"]
        extraindex = newindex + ["level_1"]
        value = "gencaptot"
        types = {}
        tabletype = "multiple"

    if parameter == "newcap":
        table = "var_new_pcap"
        index = ["g"]
        newindex = ["technology"]
        extraindex = newindex + ["level_1"]
        value = "gencapnew"
        types = {}
        tabletype = "multiple"

    if parameter == "gen":
        table = "var_gen"
        index = ["h", "z", "g"]
        newindex = ["hour", "zone", "technology"]
        extraindex = newindex + ["level_3"]
        value = "genamttot"
        types = {"hour": int}
        tabletype = "multiple"

    if parameter == "gentot":
        table = "o_gen_tot"
        index = ["g"]
        newindex = ["technology"]
        extraindex = newindex
        value = "genamttot"
        types = {}
        tabletype = "single"

    if parameter == "pgen":
        table = "var_pgen"
        index = ["h", "z"]
        newindex = ["hour", "zone"]
        extraindex = newindex + ["level_2"]
        value = "pgen"
        types = {"hour": int}
        tabletype = "multiple"

    if parameter == "pgenz":
        table = "o_pgen_tot_z"
        index = ["z"]
        newindex = ["zone"]
        extraindex = newindex
        value = "pgen"
        types = {}
        tabletype = "single"

    if parameter == "demand":
        table = "demand"
        index = ["z", "h"]
        newindex = ["zone", "hour"]
        extraindex = newindex
        value = "demand"
        types = {"hour": int}
        tabletype = "single"

    if parameter == "area":
        table = "area"
        index = ["vre", "z", "r"]
        newindex = ["technology", "zone", "region"]
        extraindex = newindex
        value = "area"
        types = {}
        tabletype = "single"

    if parameter == "storage_pcap":
        table = "var_tot_store_pcap"
        index = ["s"]
        newindex = ["technology"]
        extraindex = newindex + ["level_1"]
        value = "storepcaptot"
        types = {}
        tabletype = "multiple"

    if parameter == "storage_ecap":
        table = "var_tot_store_ecap"
        index = ["s"]
        newindex = ["technology"]
        extraindex = newindex + ["level_1"]
        value = "storeecaptot"
        types = {}
        tabletype = "multiple"

    if parameter == "storage_gen_tot":
        table = "o_store_gen_all"
        index = ["s"]
        newindex = ["technology"]
        extraindex = newindex
        value = "storegentot"
        types = {}
        tabletype = "single"

    if parameter == "vre_gen":
        table = "vre_gen"
        index = ["h", "vre", "r"]
        newindex = ["hour", "technology", "region"]
        extraindex = newindex
        value = "capfac"
        types = {}
        tabletype = "single"

    if parameter == "costsgencapex":
        table = "costs_gen_capex"
        index = ["z"]
        newindex = ["zone"]
        extraindex = newindex + ["level_1"]
        value = "costsgencapex"
        types = {}
        tabletype = "multiple"

    if parameter == "costsgenvarom":
        table = "costs_gen_varom"
        index = ["z"]
        newindex = ["zone"]
        extraindex = newindex + ["level_1"]
        value = "costsgenvarom"
        types = {}
        tabletype = "multiple"

    if parameter == "limpcapz":
        table = "gen_lim_pcap_z"
        index = ["z","g","lt"]
        newindex = ["zone","technology","limittype"]
        extraindex = newindex
        value = "limpcapz"
        types = {}
        tabletype = "single"

    if parameter == "gridcap":
        table = "o_vre_cap_z_sum"
        index = ["vre", "r"]
        newindex = ["technology", "region"]
        extraindex = newindex
        value = "gridcap"
        types = {}
        tabletype = "single"

    # if parameter == "pcap:
    #    table = "var_

    indexlist = basicindexlist + extraindex
    indexdict = dict(zip(index, newindex))
    indexdict.update({0: value})
    for scenario in tqdm(scenariolist):
        # for scenario in scenariolist:
        scenarioname = str(scenario).split("/")[-2]
        year = str(scenario).split("/")[-3]
        #    scenarioname = str(scenario)[18:-11]
        scenarionamesplit = scenarioname.split("_")

        con = sqlite3.connect(scenario)

        appendme = pd.read_sql_query(f"SELECT * from {table}", con)

        if tabletype == "multiple":
            appendme = appendme.set_index(index).stack().reset_index()

        appendme = (
            appendme.rename(columns=indexdict)
            .assign(
                year=year,
                Nature=scenarionamesplit[0],
                Fauna=scenarionamesplit[1],
                Sami=scenarionamesplit[2],
                Neigh=scenarionamesplit[3],
                Solar=scenarionamesplit[4],
                Spatial=scenarionamesplit[5],
                Cutoff=scenarionamesplit[6]
            )
            .astype(types)
            .set_index(indexlist)
            .sort_index()
        )
        returnlist.append(appendme)
        con.close()
    return pd.concat(returnlist)

In [ ]:
#sqlite_list = found_scenarios

In [ ]:
#sqlite_list = requested_scenarios.query("status != 'optimal'").path / "results.db"

In [ ]:
sqlite_list = requested_scenarios.query("status == 'optimal' and spatials == 'grid'").path / "results.db"

In [ ]:
def barplot(inputdf,logx=False):
    return (
        inputdf.groupby(
            ["year", "Nature", "Fauna", "Sami", "Neigh", "Solar", "Spatial","Cutoff"]
        )
        .sum()
        .reset_index()
        .assign(
            name=lambda df: df.year.astype(str)
            + "_"
            + df.Nature
            + "_"
            + df.Fauna
            + "_"
            + df.Sami
            + "_"
            + df.Neigh
            + "_"
            + df.Solar
            + "_"
            + df.Spatial
            + "_"
            + df.Cutoff
        )
        .set_index("name")
        .sort_index()
        # .sort_values(by="value")
        .loc[:, "value"]
        .plot.barh(figsize=(10, 15), logx=logx)
    )

## Area

In [ ]:
# How much area is available per scenario for onshore wind
area = makenice(sqlite_list, "area")

In [ ]:
# https://docs.xarray.dev/en/latest/user-guide/plotting.html#id4
(
    area.query("technology == 'Windonshore'")
    .reset_index()
    .assign(
        x1_coord=lambda df: (
            df.region.str.split("x").str[1].str.split("y").str[0].str[:-2]
        ),
        x2_coord=lambda df: (
            df.region.str.split("x").str[1].str.split("y").str[0].str[-2:]
        ),
        x_coord=lambda df: df.x1_coord + "." + df.x2_coord,
        y1_coord=lambda df: df.region.str.split("y").str[1].str[:-2],
        y2_coord=lambda df: df.region.str.split("y").str[1].str[-2:],
        y_coord=lambda df: df.y1_coord + "." + df.y2_coord,
    )
    .astype({"x_coord": float, "y_coord": float})
    .groupby(
        [
            "year",
            "Nature",
            "Fauna",
            "Sami",
            "Neigh",
            "Solar",
            "Spatial",
            "Cutoff",
            "x_coord",
            "y_coord",
        ]
    )
    .sum()
    .reset_index()
    .assign(
        name=lambda df: 
        df.year.astype(str).str[-2:]
        + "_"
        + df.Nature.str[0]
        + "_"
        + df.Fauna.str[0]
        + "_"
        + df.Sami.str[0]
        + "_"
        + df.Neigh.str[0]
        + "_"
        + df.Solar.str[0]
        + "_"
        + df.Spatial.str[0]
        + "_"
        + df.Cutoff.str[0]
    )
    .set_index(["name", "x_coord", "y_coord"])
    .sort_index()
    .loc[:, "value"]
    .to_xarray()
    .plot(x="x_coord", y="y_coord", col="name", col_wrap=5)
)

### Grid cells

In [ ]:
grid_area_start = (
    area
    .reset_index()
    .assign(
        name=lambda df: df.year.astype(str)
        + "_"
        + df.Nature
        + "_"
        + df.Fauna
        + "_"
        + df.Sami
        + "_"
        + df.Neigh
        + "_"
        + df.Solar
        + "_"
        + df.Spatial
        + "_"
        + df.Cutoff
    )
    .set_index("name")
    .sort_index()
    .query("technology == 'Windonshore'")
)

In [ ]:
nr_scenarios = len(grid_area_start.reset_index().name.unique())
nr_columns = 1
nr_rows = nr_scenarios
fig, axes = plt.subplots(nr_rows,nr_columns,figsize=(10,100))
count = 0
for scenario in tqdm(grid_area_start.reset_index().name.unique()):
    grid_area = (
        grid_area_start
        .loc[scenario]
        .loc[:,['region','value']]
        .assign(
            x1_coord = lambda df : (
                df.region.str.split("x").str[1].str.split("y").str[0].str[:-2]),
            x2_coord = lambda df : (
                df.region.str.split("x").str[1].str.split("y").str[0].str[-2:]),
            x_coord = lambda df : df.x1_coord + "." + df.x2_coord,
            y1_coord = lambda df : df.region.str.split("y").str[1].str[:-2],
            y2_coord = lambda df : df.region.str.split("y").str[1].str[-2:],
            y_coord = lambda df : df.y1_coord + "." + df.y2_coord
        )
        .astype({"x_coord":float,"y_coord":float})
        .groupby((['y_coord','x_coord'])).sum()
        .loc[:,'value']
        .to_xarray()
    )

    grid_area.plot(ax=axes[count],zorder=1,cmap='OrRd')
    norway.plot(ax=axes[count],zorder=5,alpha=0.1)
    
    axes[count].set_xlabel('Longitude')
    axes[count].set_ylabel('Latitude')
    axes[count].set_xlim(4,32)
    axes[count].set_ylim(57.5,71.5)
    axes[count].set_title(scenario + '\n Total area: ' + str(grid_area.sum().values),fontsize=8)
    count += 1 

In [ ]:
fig, ax = plt.subplots(dpi=150)

grid_area.plot(ax=ax,zorder=1,cmap='OrRd')
norway.plot(ax=ax,zorder=5,alpha=0.1)

ax.set_xlabel('Longitude')
ax.set_xlim(4,32)
ax.set_ylim(57.5,71.5)

### Via Atlite

In [ ]:
#masked, transform = shape_availability(croatia, excluder)
#eligible_share = masked.sum() * excluder.res**2 / croatia.geometry.item().area
#
#fig, ax = plt.subplots()
#ax = show(masked, transform=transform, cmap='Greens', ax=ax)
#croatia.plot(ax=ax, edgecolor='k', color='None')
#ax.set_title(f'Eligible area (green) {eligible_share * 100:2.2f}%');


## Capacity factors

In [ ]:
#capfacs = makenice(sqlite_list, "vre_gen")# Crashed computer


In [ ]:
scenario_year = scenario[0:4]

In [ ]:
scenario[5:]

In [ ]:
capfacs = (
    pd
    .read_csv(
        resultspath / 'models/2010/High_Low_High_Low_Low_grid_0.0/capacity-factors_wind_norway-g_2010.csv',
        header=None
    )
    .rename(columns={0 : 'hour', 1 : 'technology', 2 : 'grid_cell', 3 : 'capfac'})
)

fix,ax = plt.subplots()

(
    capfacs
    .groupby(['technology','grid_cell']).mean()
    .loc[:,'capfac']
    .to_frame()
    .reset_index()
    .assign(
            x1_coord = lambda df : (
                df.grid_cell.str.split("x").str[1].str.split("y").str[0].str[:-2]),
            x2_coord = lambda df : (
                df.grid_cell.str.split("x").str[1].str.split("y").str[0].str[-2:]),
            x_coord = lambda df : df.x1_coord + "." + df.x2_coord,
            y1_coord = lambda df : df.grid_cell.str.split("y").str[1].str[:-2],
            y2_coord = lambda df : df.grid_cell.str.split("y").str[1].str[-2:],
            y_coord = lambda df : df.y1_coord + "." + df.y2_coord
        )
    .astype({"x_coord":float,"y_coord":float})
    .groupby((['y_coord','x_coord'])).sum()
    .loc[:,'capfac']
    .to_xarray()
).plot(ax=ax)

norway.boundary.plot(ax=ax)


## Installed capacity

In [ ]:
gridcap = makenice(sqlite_list, "gridcap")

In [ ]:
gridcap = (
    gridcap.reset_index()
    .assign(
        name=lambda df: df.year.astype(str)
        + "_"
        + df.Nature
        + "_"
        + df.Fauna
        + "_"
        + df.Sami
        + "_"
        + df.Neigh
        + "_"
        + df.Solar
        + "_"
        + df.Spatial
        + "_"
        + df.Cutoff
    )
    .set_index("name")
    .sort_index()
    .query("technology == 'Windonshore'")
)

## Generation

In [ ]:
gentot = makenice(sqlite_list,"gentot")

In [ ]:
(
    gentot
    .reset_index()
    .assign(
        name=lambda df: df.year.astype(str)
        + "_"
        + df.Nature
        + "_"
        + df.Fauna
        + "_"
        + df.Sami
        + "_"
        + df.Neigh
        + "_"
        + df.Solar
        + "_"
        + df.Spatial
        + "_"
        + df.Cutoff
    )
    .set_index("name")
    .sort_index()
    .query("technology == 'Windonshore'")
    .assign(
        capacity = gridcap.groupby('name').sum(),
        avg_cp = lambda x : (x.value) / (x.capacity*8760),
    )
)

## Combined area and installed cap

In [ ]:
(
    grid_area_start
    .loc[scenario]
    .loc[:,['region','value']]
    .assign(
        x1_coord = lambda df : (
            df.region.str.split("x").str[1].str.split("y").str[0].str[:-2]),
        x2_coord = lambda df : (
            df.region.str.split("x").str[1].str.split("y").str[0].str[-2:]),
        x_coord = lambda df : df.x1_coord + "." + df.x2_coord,
        y1_coord = lambda df : df.region.str.split("y").str[1].str[:-2],
        y2_coord = lambda df : df.region.str.split("y").str[1].str[-2:],
        y_coord = lambda df : df.y1_coord + "." + df.y2_coord
    )
    .astype({"x_coord":float,"y_coord":float})
    .groupby((['y_coord','x_coord'])).sum()
    .loc[:,'value']
    .to_xarray()
    .plot(cmap='OrRd',levels=np.linspace(0, grid_area_start.value.max(), 6, grid_area_start.value.max()))
)

In [ ]:
gridcap.value.max()

In [ ]:
grid_area_start.value.max()

In [ ]:
#cap2area = 3 #MW/km2
nr_scenarios = len(grid_area_start.reset_index().name.unique())
nr_columns = 2
nr_rows = nr_scenarios
fig, axes = plt.subplots(nr_rows,nr_columns,figsize=(10,200))
fig.tight_layout()
row = 0
column = 0
levels_1 = np.linspace(0, grid_area_start.value.max(), 10, grid_area_start.value.max())
levels_2 = np.linspace(0, gridcap.value.max(), 10, gridcap.value.max())

for scenario in tqdm(grid_area_start.reset_index().name.unique()):
    column = 0
    (
        grid_area_start
        .loc[scenario]
        .loc[:,['region','value']]
        .assign(
            x1_coord = lambda df : (
                df.region.str.split("x").str[1].str.split("y").str[0].str[:-2]),
            x2_coord = lambda df : (
                df.region.str.split("x").str[1].str.split("y").str[0].str[-2:]),
            x_coord = lambda df : df.x1_coord + "." + df.x2_coord,
            y1_coord = lambda df : df.region.str.split("y").str[1].str[:-2],
            y2_coord = lambda df : df.region.str.split("y").str[1].str[-2:],
            y_coord = lambda df : df.y1_coord + "." + df.y2_coord
        )
        .astype({"x_coord":float,"y_coord":float})
        .groupby((['y_coord','x_coord'])).sum()
        .loc[:,'value']
        .to_xarray()
        .plot(ax=axes[row,column],zorder=1,cmap='OrRd',levels=levels_1)
    )

    norway.boundary.plot(ax=axes[row,column],zorder=5,alpha=0.1)
    
    axes[row,column].set_xlabel('Longitude')
    axes[row,column].set_ylabel('Latitude')
    axes[row,column].set_xlim(4,32)
    axes[row,column].set_ylim(57.5,71.5)
    axes[row,column].set_title(
        scenario 
        + '\n Total area: ' 
        + str(round(grid_area_start.loc[scenario,['value']].value.sum(),3))
        ,fontsize=8)
    column += 1 

    (
        gridcap
        .query("region != @desired_regions")
        .loc[scenario]
        .loc[:,['region','value']]
        .assign(
            x1_coord = lambda df : (
                df.region.str.split("x").str[1].str.split("y").str[0].str[:-2]),
            x2_coord = lambda df : (
                df.region.str.split("x").str[1].str.split("y").str[0].str[-2:]),
            x_coord = lambda df : df.x1_coord + "." + df.x2_coord,
            y1_coord = lambda df : df.region.str.split("y").str[1].str[:-2],
            y2_coord = lambda df : df.region.str.split("y").str[1].str[-2:],
            y_coord = lambda df : df.y1_coord + "." + df.y2_coord
        )
        .astype({"x_coord":float,"y_coord":float})
        .groupby((['y_coord','x_coord'])).sum()
        .loc[:,'value']
        .to_xarray()
        .plot(ax=axes[row,column],zorder=1,cmap='OrRd',levels=levels_2)
    )

    norway.boundary.plot(ax=axes[row,column],zorder=5,alpha=0.1)

    axes[row,column].set_xlabel('Longitude')
    axes[row,column].set_ylabel('Latitude')
    axes[row,column].set_xlim(4,32)
    axes[row,column].set_ylim(57.5,71.5)
    axes[row,column].set_title(scenario 
        + '\n Installed capacity: ' 
        + str(round((gridcap.query("region != @desired_regions").loc[scenario].value.sum()),3))
        + ' GW'
        ,fontsize=8)
        
    
    row += 1
    
fig.subplots_adjust(wspace=0.1)
#plt.savefig('scenario_plot_AreaCap.pdf',bbox_inches='tight')
#plt.savefig('scenario_plot_AreaCap.png',bbox_inches='tight')
#plt.savefig('scenario_plot_AreaCap.svg',bbox_inches='tight')

## Combined area and capfac

In [ ]:
nr_scenarios = len(grid_area_start.reset_index().name.unique())
nr_columns = 2
nr_rows = nr_scenarios
fig, axes = plt.subplots(nr_rows,nr_columns,figsize=(10,10))
row = 0
column = 0
for scenario in tqdm(grid_area_start.reset_index().name.unique()):
    column = 0
    grid_area = (
        grid_area_start
        .loc[scenario]
        .loc[:,['region','value']]
        .assign(
            x1_coord = lambda df : (
                df.region.str.split("x").str[1].str.split("y").str[0].str[:-2]),
            x2_coord = lambda df : (
                df.region.str.split("x").str[1].str.split("y").str[0].str[-2:]),
            x_coord = lambda df : df.x1_coord + "." + df.x2_coord,
            y1_coord = lambda df : df.region.str.split("y").str[1].str[:-2],
            y2_coord = lambda df : df.region.str.split("y").str[1].str[-2:],
            y_coord = lambda df : df.y1_coord + "." + df.y2_coord
        )
        .astype({"x_coord":float,"y_coord":float})
        .groupby((['y_coord','x_coord'])).sum()
        .loc[:,'value']
        .to_xarray()
    )

    grid_area.plot(ax=axes[row,column],zorder=1,cmap='OrRd')
    norway.plot(ax=axes[row,column],zorder=5,alpha=0.1)
    
    axes[row,column].set_xlabel('Longitude')
    axes[row,column].set_ylabel('Latitude')
    axes[row,column].set_xlim(4,32)
    axes[row,column].set_ylim(57.5,71.5)
    axes[row,column].set_title(scenario + '\n Total area: ' + str(grid_area.sum().values),fontsize=8)
    column += 1 

    capfacs = (
        pd.read_csv(
            '/home/oskar/Desktop/uio/data/local_joint_wind/models/' 
            + scenario_year 
            + '/'
            + scenario[5:] 
            + '/capacity-factors_wind_norway-g_2010.csv',
            header=None
        )
        .rename(columns={0 : 'hour', 1 : 'technology', 2 : 'grid_cell', 3 : 'capfac'})
    )

    (
        capfacs
            .groupby(['technology','grid_cell']).mean()
            .loc[:,'capfac']
            .to_frame()
            .reset_index()
            .assign(
                x1_coord = lambda df : (
                    df.grid_cell.str.split("x").str[1].str.split("y").str[0].str[:-2]),
                x2_coord = lambda df : (
                    df.grid_cell.str.split("x").str[1].str.split("y").str[0].str[-2:]),
                x_coord = lambda df : df.x1_coord + "." + df.x2_coord,
                y1_coord = lambda df : df.grid_cell.str.split("y").str[1].str[:-2],
                y2_coord = lambda df : df.grid_cell.str.split("y").str[1].str[-2:],
                y_coord = lambda df : df.y1_coord + "." + df.y2_coord
            )
            .astype({"x_coord":float,"y_coord":float})
            .groupby((['y_coord','x_coord'])).sum()
            .loc[:,'capfac']
            .to_xarray()
            .plot(ax=axes[row,column],zorder=1,cmap='OrRd')
    )

    norway.boundary.plot(ax=axes[row,column],zorder=5,alpha=0.1)

    axes[row,column].set_xlabel('Longitude')
    axes[row,column].set_ylabel('Latitude')
    axes[row,column].set_xlim(4,32)
    axes[row,column].set_ylim(57.5,71.5)
    axes[row,column].set_title(scenario + '\n Mean capacity factor: ' + str(round(capfacs.groupby(['technology','grid_cell']).mean().drop(columns={'hour'}).capfac.mean(),3)),fontsize=8)
    
    row += 1

plt.savefig('scenario_plot_AreaCapfac.pdf',bbox_inches='tight')
plt.savefig('scenario_plot_AreaCapfac.png',bbox_inches='tight')
plt.savefig('scenario_plot_AreaCapfac.svg',bbox_inches='tight')

In [ ]:
round(capfacs.groupby(['technology','grid_cell']).mean().drop(columns={'hour'}).capfac.mean(),3)